# Ручка 3. Summary

Делает краткое саммари всего конспекта (на вход идет весь файл в обсидиане, можно добавить передачу всей папки)

In [3]:
from langchain.chat_models.gigachat import GigaChat
from dotenv import load_dotenv
import os

load_dotenv()

GIGACHAT_CREDENTIALS = os.environ.get("GIGACHAT_CREDENTIALS")

llm = GigaChat(
    credentials=GIGACHAT_CREDENTIALS,
    model="GigaChat-Pro",
    verify_ssl_certs=True,
    scope="GIGACHAT_API_CORP",
)

In [4]:
# Загружаем конспект
from langchain_community.document_loaders.text import TextLoader

notes_path = "./notes/small_note.md"

loader = TextLoader(notes_path, "utf-8")
docs = loader.load()

print(docs[0].page_content)

# Используем Stuff Documents Chain + Summarization Checker

In [3]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMSummarizationCheckerChain

def stuff_summary(docs, checker: bool=False, max_checks:int=2):
     prompt_template = """Выдели клюбючевые моменты в тексте.
     Оформи ответ в виде списка.
     Текст: "{context}"
     Ключевые моменты:"""
     prompt = PromptTemplate.from_template(prompt_template)
     stuff_chain = create_stuff_documents_chain(llm, prompt)
     res = stuff_chain.invoke({"context": docs})
     if not checker:
          return res
     else:
          checker_chain = LLMSummarizationCheckerChain.from_llm(llm, max_checks=max_checks)
          return checker_chain.run(res)
          

print(stuff_summary(docs, True, 2))

c:\Users\nonam\Documents\python_local\obsidian-gigachat\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


- Влияние параметров пара на показатели комбинированной установки с простым паровым контуром
- Максимальная мощность паровой турбины достигается при максимальных параметрах пара
- Мощность паровой турбины зависит от расхода пара, располагаемого перепада энтальпий на турбину и КПД паровой турбины
- Относительный расход пара определяется уравнением теплового баланса испарительной и перегревательной поверхностей котла-утилизатора
- Температура пара перед паровой турбиной принимается максимально возможной для уменьшения потерь от влажности и повышения КПД турбины
- С увеличением давления пара относительный расход пара снижается из-за снижения удельной теплоты парообразования
- С ростом давления растет температура уходящих газов и располагаемый перепад энтальпий на паровую турбину
- Совместное влияние параметров на величину мощности паровой турбины приводит к слабой зависимости КПД парогазовой установки от начального давления
- Температура уходящих газов остается выше минимально допустимой,

# Используем Map-Reduce

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter


# Map
map_template = """Ниже представлен список документов
{docs}
На основе этих документов определи, пожалуйста, основные темы 
Полезный ответ:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

reduce_template = """Ниже представлен набор кратких изложений:
{docs}
Возьми их и сведи в окончательное, объединенное краткое изложение основных тем. 
Полезный ответ:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=8000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

map_reduce_chain.run(split_docs)

'Основные темы, представленные в документах, включают влияние параметров пара на показатели комбинированной установки с простым паровым контуром, зависимость мощности паровой турбины от расхода пара, располагаемого перепада энтальпий на турбину и КПД паровой турбины, определение относительного расхода пара через уравнение теплового баланса испарительной и перегревательной поверхностей котла-утилизатора, влияние давления пара на относительный расход пара и температуру уходящих газов, а также влияние температуры пара перед паровой турбиной на влажность и КПД турбины.'

# 2 Поиск терминов

In [15]:
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.question_answering import load_qa_chain

note_path = "./notes/water_treatment.md"

loader = TextLoader(note_path, "utf-8")
docs = loader.load()

prompt_template = """Твоя задача найти в тексте термины без определений.
Игнорируй обычные слова. Ищи только термины, определения и сокращения, которым не хватает расшифровки.
Если таких нет, ничего не возвращай.
Текст: "{context}"
Ответ:"""
# prompt = PromptTemplate.from_template(prompt_template)
# stuff_chain = create_stuff_documents_chain(llm, prompt)
# stuff_chain.invoke({"context": docs})

qa_chain = load_qa_chain(llm, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
print(qa_document_chain.run(
    input_document=docs[0].page_content,
    question="""Найди в тексте сокращения. Выведи их в формате ['термин_1', 'термин_2',
    'термин_3', 'термин_4']. Не мотивируй свой ответ, не выводи другой текст.""",
))

['SDR', 'GR-LTE', 'srsUE', 'srsRAN', 'eNodeB', 'USRP', 'PC/SC']
